In [1]:
import pandas as pd
from country_list import countries_for_language
import geonamescache

In [41]:
df_hotel = pd.read_parquet("./raw/0000.parquet")
df_travel = pd.read_csv("./raw/Travel details dataset.csv")
df_travel = df_travel.dropna(how="all")
df_car = pd.read_csv("./raw/CarRentalData.csv")
df_flight = pd.read_csv("./raw/Air_Traffic_Passenger_Statistics.csv")
df_airbnb = pd.read_csv("./raw/airnb.csv")

hotel data manipulation

In [7]:
split_data = df_hotel['city'].str.split(", ")
df_hotel['country'] = split_data.apply(lambda x: x[1] if len(x) > 1 else x[0])
df_hotel['city'] = split_data.apply(lambda x: x[0] if len(x) > 1 else None)

In [8]:
df_hotel.to_csv("./data/df_hotel.csv")

In [ ]:
res = []
for h in df_hotel.city:
    res.append(h.split(", "))
res

travel data manipulation

In [3]:
def is_country(name):
    # Check if name matches a country name
    countries = dict(countries_for_language('en'))
    return name in countries.values()

def get_country_from_city(city_name):
    gc = geonamescache.GeonamesCache()
    cities = gc.get_cities_by_name(city_name)

    # If city is found and the data is available
    if cities and cities[0]:
        city_data = cities[0]
        city_key = list(city_data.keys())[0]
        country_code = city_data[city_key]['countrycode']

        # Check if country code is in the countries dataset
        countries = gc.get_countries()
        if country_code in countries:
            return countries[country_code]['name']
    return None

In [42]:
def destinationDecomposition(destination):
    city, country = None, None
    if "," in destination:
        city, country = destination.split(",")
    else:
        if is_country(destination):
            country = destination
        else:
            city = destination
            country = get_country_from_city(city)
            if city == "New York":
                country = "USA"
    return city, country
df_travel['Destination_City'], df_travel['Destination_Country'] = zip(*df_travel["Destination"].apply(destinationDecomposition))
df_travel['Destination_Country'] = df_travel['Destination_Country'].str.lstrip()
df_travel['Accommodation cost'] = df_travel['Accommodation cost'].str.replace('$', '').replace(',', '')
df_travel['Transportation cost'] = df_travel['Transportation cost'].str.replace('$', '').replace(',', '')

C:\Users\26871\AppData\Local\Temp\ipykernel_2436\3406990357.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_travel['Accommodation cost'] = df_travel['Accommodation cost'].str.replace('$', '').replace(',', '')
C:\Users\26871\AppData\Local\Temp\ipykernel_2436\3406990357.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_travel['Transportation cost'] = df_travel['Transportation cost'].str.replace('$', '').replace(',', '')


In [5]:
df_travel.head()

,Trip ID,Destination,Start date,End date,Duration (days),Traveler name,Traveler age,Traveler gender,Traveler nationality,Accommodation type,Accommodation cost,Transportation type,Transportation cost,Destination_City,Destination_Country
0,1,"London, UK",5/1/2023,5/8/2023,7,John Smith,35,Male,American,Hotel,1200,Flight,600,London,UK
1,2,"Phuket, Thailand",6/15/2023,6/20/2023,5,Jane Doe,28,Female,Canadian,Resort,800,Flight,500,Phuket,Thailand
2,3,"Bali, Indonesia",7/1/2023,7/8/2023,7,David Lee,45,Male,Korean,Villa,1000,Flight,700,Bali,Indonesia
3,4,"New York, USA",8/15/2023,8/29/2023,14,Sarah Johnson,29,Female,British,Hotel,2000,Flight,1000,New York,USA
4,5,"Tokyo, Japan",9/10/2023,9/17/2023,7,Kim Nguyen,26,Female,Vietnamese,Airbnb,700,Train,200,Tokyo,Japan


In [8]:
def manipulate_data(df, column_name, old_value, new_value):
    df.loc[df[column_name] == old_value, column_name] = new_value
    return df

In [43]:
# destination city uniformization
df_travel = manipulate_data(df_travel, "Destination_City", "New York", "New York City")
df_travel = manipulate_data(df_travel, "Destination_City", "Honolulu", "Hawaii")

In [44]:
# destination country uniformization
df_travel = manipulate_data(df_travel, "Destination_Country", "United Kingdom", "UK")
df_travel = manipulate_data(df_travel, "Destination_Country", "Aus", "Australia")
df_travel = manipulate_data(df_travel, "Destination_Country", "AUS", "Australia")
df_travel = manipulate_data(df_travel, "Destination_Country", "SA", "South Africa")
df_travel = manipulate_data(df_travel, "Destination_Country", "Hawaii", "USA")
df_travel = manipulate_data(df_travel, "Destination_Country", "Thai", "Thailand")

In [48]:
# traveler nationality uniformization
df_travel = manipulate_data(df_travel, "Traveler nationality", "UK", "British")
df_travel = manipulate_data(df_travel, "Traveler nationality", "United Kingdom", "British")
df_travel = manipulate_data(df_travel, "Traveler nationality", "USA", "American")
df_travel = manipulate_data(df_travel, "Traveler nationality", "Canada", "Canadian")
df_travel = manipulate_data(df_travel, "Traveler nationality", "Korean", "South Korean")
df_travel = manipulate_data(df_travel, "Traveler nationality", "South Korea", "South Korean")
df_travel = manipulate_data(df_travel, "Traveler nationality", "Australia", "Australian")
df_travel = manipulate_data(df_travel, "Traveler nationality", "Netherlands", "Dutch")
df_travel = manipulate_data(df_travel, "Traveler nationality", "Brazil", "Brazilian")
df_travel = manipulate_data(df_travel, "Traveler nationality", "United Arab Emirates", "Emirati")
df_travel = manipulate_data(df_travel, "Traveler nationality", "Mexico", "Mexican")
df_travel = manipulate_data(df_travel, "Traveler nationality", "Japan", "Japanese")
df_travel = manipulate_data(df_travel, "Traveler nationality", "China", "Chinese")
df_travel = manipulate_data(df_travel, "Traveler nationality", "Spain", "Spanish")
df_travel = manipulate_data(df_travel, "Traveler nationality", "Germany", "German")
df_travel = manipulate_data(df_travel, "Traveler nationality", "Taiwan", "Taiwanese")
df_travel = manipulate_data(df_travel, "Traveler nationality", "Italy", "Italian")
df_travel = manipulate_data(df_travel, "Traveler nationality", "South Africa", "South African")
df_travel = manipulate_data(df_travel, "Traveler nationality", "Hong Kong", "Chinese")

In [49]:
df_travel["Traveler nationality"].unique()

array(['American', 'Canadian', 'South Korean', 'British', 'Vietnamese',
       'Australian', 'Brazilian', 'Dutch', 'Emirati', 'Mexican',
       'Spanish', 'Chinese', 'German', 'Moroccan', 'Scottish', 'Japanese',
       'Italian', 'Indian', 'French', 'South African', 'Taiwanese',
       'Indonesian', ' African', 'Singapore', 'Greece', 'Cambodia',
       'New Zealander'], dtype=object)

In [50]:
df_travel = df_travel.drop('Destination', axis=1)
df_travel.to_csv('./data/df_travel_new.csv', index=False)

airbnb data manipulation

In [9]:
split_data = list(map(lambda x: x.split(", "), df_airbnb['Title']))

# Initialize empty lists for each column
titles = []
states_or_countries = []
countries = []

# Step 2: Iterate through the split_data
for entry in split_data:
    titles.append(entry[0])  # The first element is always the title
    
    if len(entry) == 1:
        states_or_countries.append(None)
        countries.append(None)
    elif len(entry) == 2:
        states_or_countries.append(None)
        countries.append(entry[1])
    elif len(entry) == 3:
        states_or_countries.append(entry[1])
        countries.append(entry[2])

# Step 3: Add these lists as new columns to df_airbnb
df_airbnb['Extracted Title'] = titles
df_airbnb['City/State'] = states_or_countries
df_airbnb['Country'] = countries


In [10]:
df_airbnb = df_airbnb.drop(["Title", "Date"], axis=1)

In [11]:
df_airbnb.to_csv("./data/df_airbnb.csv")

flight data manipulation

In [12]:
df_flight = df_flight[['Operating Airline',
       'Operating Airline IATA Code', 'Published Airline',
       'Published Airline IATA Code', 'GEO Summary', 'GEO Region',
       'Price Category Code', "Terminal", "Boarding Area"
       ]]

In [13]:
df_flight.drop_duplicates().to_csv("./data/df_flight.csv")

In [59]:
set(df_flight['GEO Region'])

{'Asia',
 'Australia / Oceania',
 'Canada',
 'Central America',
 'Europe',
 'Mexico',
 'Middle East',
 'South America',
 'US'}

car rental data manipulation

In [14]:
df_car = df_car[['fuelType', 'rating', 'renterTripsTaken', 'reviewCount', 
                'owner.id','vehicle.make', 'vehicle.model',
                'vehicle.type', 'vehicle.year']]

In [15]:
df_car.to_csv("./data/df_car.csv")